# Cloudwatch Metrics に学習過程のスコアを書き出す

## 概要

このノートブックでは，Amazon SageMaker 上で学習する際のスコアを，Cloudwatch Metrics に書き出して可視化するやり方について確認します．


## データセットのS3へのアップロード

- keras.datasetsを利用してmnistのデータをダウンロードしてnpz形式で保存します。
- 保存したnpz形式のファイルを、SageMaker Python SDKを利用してS3にアップロードします。

In [ ]:
import os
import keras
import numpy as np
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/train', image=x_train, label=y_train)
np.savez('./data/test', image=x_test, label=y_test)

import sagemaker
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
input_data = sagemaker_session.upload_data(path='./data', bucket=bucket_name, key_prefix='dataset/mnist')
print('Training data is uploaded to: {}'.format(input_data))

## メトリクスの記述

Estimator オブジェクトを作成する際に，metric_definitions を JSON 形式で指定することができます．ここで正規表現の形でメトリクスを指定することで，ジョブを実行した際の標準出力からマッチする値を取り出して，自動的に Cloudwatch Metrics に書き出してくれます．

ここでは Keras のジョブを実行するため，以下のような形式でログが出力されます．エポックごとの，訓練データと評価データ両方に対する損失関数の値を，メトリクスとして抜き出すことを考えてみましょう．

```
59600/60000 [============================>.] - ETA: 0s - loss: 0.2289 - acc: 0.9298
59800/60000 [============================>.] - ETA: 0s - loss: 0.2286 - acc: 0.9299
60000/60000 [==============================] - 28s 460us/step - loss: 0.2282 - acc: 0.9300 - val_loss: 0.1047 - val_acc: 0.9671
Epoch 2/100

  100/60000 [..............................] - ETA: 28s - loss: 0.1315 - acc: 0.9500
  300/60000 [..............................] - ETA: 25s - loss: 0.1260 - acc: 0.9600
  500/60000 [..............................] - ETA: 25s - loss: 0.1209 - acc: 0.9620
```

ここでは，以下のようにメトリクスを定義することで，上記形式のログから，訓練・評価データそれぞれの損失関数の値を抜き出すことができます．

```
          metric_definitions=[
              { "Name": "train:loss", "Regex": ".*step\\s-\\sloss:\\s(\\S+).*" },
              { "Name": "val:loss", "Regex": ".*\\sval_loss:\\s(\\S+).*" }
          ],
```


## SageMakerでの学習

先ほど説明したメトリクス定義を含めて Tensorflow オブジェクトを作成し，実行することで，メトリクスも出力されます．確認のために，マネジメントコンソールの左メニュー内「トレーニングジョブ」から，該当するジョブを選択します．詳細画面の下側にある「モニタリング」フィールド内の「アルゴリズムメトリクスの表示」リンクから，定義したメトリクスのグラフに飛ぶことができます．

In [ ]:
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

role = get_execution_role()
mnist_estimator = TensorFlow(entry_point = "./src/keras_mlp_mnist.py",
          role=role,
          train_instance_count=1,
          train_instance_type="ml.m4.xlarge",
          framework_version="1.11.0",
          py_version='py3',
          script_mode=True,
          metric_definitions=[
              { "Name": "train:loss", "Regex": ".*step\\s-\\sloss:\\s(\\S+).*" },
              { "Name": "val:loss", "Regex": ".*\\sval_loss:\\s(\\S+).*" }
          ],
          hyperparameters={'batch_size': 64,
                         'n_class': 10,
                         'epochs': 15})

mnist_estimator.fit(input_data)